#BEVFormer


## Abstract

This paper covers how with spatial, temporal and camera data, one can create a Bird's Eye View of the vehicle and the objects around it using the self attention mechanism of the transformer.

Creating a BEV from 2-D and spatial data, gives a near accurate presentation of how the objects should interact. A transfromer encoder and decoder is used here and with the right queries, we can get a wide range of embeddings whihc correspond to the particular detail we want.

For our use case, generating a BEV is crucial for determining Lane topology, how the lane's interact with each other. Cetnerline is a crucial aspect as all other lanes are hevily dependent on determining the centerline coordinates.

In the open lane dataset, we have mannual annodations on data which gives us the respective cetenrline and other related lanes that will can be used by the pov vehicle. The dataset also provides us with other metrics such as area and trafiic elements.

For this paper, the dataset being used is teh NuScenes dataset and the main idea here is to use the self attention mechanism of transformers to generate spatial cross attention and also the temporal attention, i.e. using the BEV at t-1 time to estiamte velocity and other metrics for the POV vehicle and others.

## Architecture

The method employed in the paper is quite straight forward.
Since a Bird's eye view is nothing but a top down view, it can be represented as a matrix of elements. We use a 2-D matrix to represent the BEV for a particular frame. A frame is a combination of all the images picked up by the cameras and the sensors at some time `t`.

### Backbone

At this frame, we generate 2-D features from all the images. Sicne 3D depth estimation with monocular images is not as acuurate, NU Scenes provides data from a combination of sensors that help PETR estimate the 3-D bounding box for any image object. These objects could be pedestrians, traffic signs, vehicles and other things. This is the first layer of the model.

Some popular backbone models are
- ResNet
- VGG
- Inception

for the purposes of this publication, the backbone they have used is `ResNet-101`.

### Multi Camera Features

Sicne the features captured here are for every image that is taken by the on vehicle camera, we extract the featues from these images. Now these features are passed on to the encoder layer of the `former` part of the BEVFormer. These multi camera features in effect with the $BEV_t-_1$ grid that we have.

# Encoder Layer

This encoder layer is where the queies, keys and values are passed to generate a new Birds eye view of the iamges obtaied. There are three important layers here which are all followed by an `Add & Normalization` layer.


## 1. Temporal Self Attention

At the start of the encoder block, this layer take in the $BEV_t-_1$ and BEV Queries Q. The purpose of this layer is to keep track of time as the car progresses through it's path. Keeping track of other road elements this way will give a breif estimate of their velocity and other factors.  In temporal
self-attention, each BEV query interacts with two features: the BEV queries at the current timestamp and the BEV features at the previous timestamp.

The first step towards temporal self attention is to be take the BEV queries Q at current timestamp t and history BEV features $BEV_t−_1$ preserved at
timestamp `t−1`, we first align $BEV_t−_1$ to Q according to ego-motion to make the features at the same grid correspond to the same real-world location.

From times `t−1` to `t`, movable objects travel in the real world with various
offsets. It is challenging to construct the precise association of the same objects between the BEV features of different times so the Temporal Self Attention layer preserves that information and uses a specific formula to derive those values. The formula can be found in the paper, it utilises a summation of Deformable Attention to generate Temporal self attention at time `t`.

In summary, compared to other methdos, where stacking of BEV's is used to determine location of objects, here the TSA layer extracts temporal features from previous BEV's to generate the new BEV resulting in much less computational cost.


## 2. Spatial Cross Attention

This is another layer used to process all the different views of the camera at once. Since there are 7 camera views, having 7 independent heads to determine 7 different regions of BEV will be computationally expensive. In this layer, The BEV query gird is interacted with only the parts a particular image contains, i.e. we know that the front center camera will only be concerned with the fron portion of the grid and it's features, hence it can be faster than traditional methods.

We first lift each query on the BEV plane to a pillar-like query, sample
$N_r$ 3D reference points from the pillar, and then project these points to 2D views. For one BEV
query, the projected 2D points can only fall on some views, and other views are not hit. Here, we
term the hit views as Vhit. After that, we regard these 2D points as the reference points of the query
Qp and sample the features from the hit views Vhit around these reference points. Finally, we perform
a weighted sum of the sampled features as the output of spatial cross-attention. The formula for the weighthed sum average involves deformable attention and has been provided in the paper.

$SCA(Qp, Ft) = \dfrac{1}{|V_{hitd}|}
\sum_{i∈Vhit}^{}
\sum_{j=1}^{N_r} DeformAttn(Qp,P(p, i, j), F^i_t
)$

We also see how the Projection Function works, raising the 2D Features.We calculate the real world location
$(x^1, y^1)$
corresponding to the query $Q_p$ located at p = (x, y) of Q as

$
x^1 = (x−
\dfrac{W}{2}) × s$

$y^1
 = (y−
\dfrac{H}{2})×s
$

H, W are the spatial shape of BEV queries, s is the size of resolution of BEV’s grids, and
(x
, y
) are the coordinates where the position of ego car is the origin.


The $x^1, y^1$ are anchor points in the 3D world space for the 2D iamge coordinates, Similarly with the help of depth information provided with datasets a $z^1$ is also defined which takes into account the height of a particular point from  origin. These are now the raised projection points representing features of the original image.

## Deformable Attention

Traditionally, calculating attention is an expensive operation. In standard attention mechanism, each query element attends to all key elements uniformly, regardless of their spatial relationships or scales. This approach can be computationally expensive and may not always capture the most relevant features for tasks like object detection or segmentation.

Defromable attention on the other hand, uses a few key changes in order to computationally accelerate the process and generate relevant results.

- Sparse Sampling : Instead of attending to all inputs uniformly, deformable attention attends to a sparse set of spatial locations. These locations are  predicted based on the content of the input features in turn focusing on the relevant areas.

- Dynamic Offsets : These are regions of interest that have a higher chance of finding what the query is looking for. This is how we decrese the computational time significantly and speed up the process.


## Datasets

- [NuScene](https://www.nuscenes.org/) : It contains 1000 scenes of roughly 20s duration each, and the key samples
are annotated at 2Hz. Each sample consists of RGB images from 6 cameras and has 360° horizontal
FOV.

- [Waymo](https://waymo.com/open/) : It contains 798 training sequences
and 202 validation sequences. Each frame provided by Waymo have
only about 252° horizontal FOV, but the provided annotated labels are 360° around the ego car.

## Results


### 3D Object Detection

For BEVFormer, the results are quite promising, It beats DETR3D(47.9% [NDS](https://nds-association.org/)) by over **9** points and around **10** points in mAP(mean Average Precision) score.
It also beats other methods such as [FCOS 3D](https://arxiv.org/abs/2104.10956) and [PGD](https://arxiv.org/abs/2107.14160) by a good margin in object detection tasks.

These results also aid in BEVFormer to get a better presepctive on how to estiamte velocity of moving objects without using any additional data. The temporal head as explained, keeps in track the data from the previous frames and calcuates moving object's velocity. This is even helpful in cases where moving objects are occluded by some obstruction.

BEVFormer is good for 3D object detection as the results for Multi Task Perception where other factors like traffic signs, lane topology etc. are involved, it gives a marginally better results which can give an edge in certain cases but we are not concerned with those as of this moment.